In [ ]:
"""
Purpose: To test the new v5 rules that have the following
changes

1) Better Width and Double Back Implementation


"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-05-28 00:53:38,923 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-28 00:53:38,926 - settings - Setting database.user to celiib
INFO - 2021-05-28 00:53:38,928 - settings - Setting database.password to newceliipass
INFO - 2021-05-28 00:53:38,940 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-05-28 00:53:38,942 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-28 00:53:38,953 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-05-28 00:53:39,343 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-28 00:53:39,346 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-28 00:53:39,348 - settings - Setting database.user to celiib
INFO - 2021-05-28 00:53:39,349 - settings - Setting database.password to newceliipass
INFO - 2021-05-28 00:53:39,349 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-28 00:53:39,352 - settings - Setting database.user to celiib
INFO - 2021-05-28 00:53:39,353 - settings - Setting database.password to newceliipass
INFO - 2021-05-28 00:53:39,353 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-28 00:53:39,753 - settings - Setting enable_python_native_blobs to True


In [3]:
#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-05-28 00:53:39,861 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-05-28 00:53:39,862 - settings - Setting database.user to celiib
INFO - 2021-05-28 00:53:39,863 - settings - Setting database.password to newceliipass
INFO - 2021-05-28 00:53:39,864 - settings - Setting enable_python_native_blobs to True
INFO - 2021-05-28 00:53:40,384 - settings - Setting enable_python_native_blobs to True


In [4]:
import validation_utils as vu
import neuron_visualizations as nviz
import neuron_utils as nru
import proofreading_utils as pru
import time
import system_utils as su

# Downloading Neurons to Test with

In [5]:
curr_table = minnie.AutoProofreadValidationScore5() * minnie.AutoProofreadValidationSegmentMap4()
curr_table

ver materialization version,nucleus_id nucleus id,segment_id the segment id in the proofread version,old_ver materialization version,split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,decomposition saved neuron object with high fidelity axon,axon_length axon length of the filtered neuron,validation_df,validation_df_ext,pre_tp,pre_tn,pre_fp,pre_fn,pre_precision,pre_recall,pre_f1,post_tp,post_tn,post_fp,post_fn,post_precision,post_recall,post_f1,run_time the amount of time to run (seconds),old_segment_id the segment id in the proofread version--,finished_dendrite whether axon was finished cleaning,finished_axon whether axon was finished cleaning,extension_type,n_somas,n_soma_mergers,n_glia,large_splits,notes
88.00,189149,864691135855890478,0.08,0,=BLOB=,369.84999619148164,=BLOB=,=BLOB=,7,160,11,0,0.3888888888888889,1.0,0.56,5281,7,39,3,0.9926691729323308,0.9994322482967449,0.9960392304790645,365.54,864691135585319036,True,True,full,1,0,0,0,
88.00,223037,864691136227167569,0.08,0,=BLOB=,63.375728490237755,=BLOB=,=BLOB=,0,450,0,0,nan,nan,nan,1,3410,3859,0,0.00025906735751295336,1.0,0.000518000518000518,328.9,864691135538120050,True,True,full,2,0,0,0,
88.00,225498,864691135753932237,0.08,0,=BLOB=,185.8554437449504,=BLOB=,=BLOB=,6,77,0,5,1.0,0.5454545454545454,0.7058823529411764,1124,0,0,9,1.0,0.9920564872021183,0.9960124058484714,126.79,864691135564655959,True,True,full,1,0,0,0,
88.00,230236,864691135609687047,0.08,0,=BLOB=,1560.5711064168272,=BLOB=,=BLOB=,34,550,11,16,0.7555555555555555,0.68,0.7157894736842104,1961,22,1,1,0.9994903160040775,0.9994903160040775,0.9994903160040775,403.91,864691135748568361,True,True,full,1,0,0,0,
88.00,236197,864691136903144370,0.08,0,=BLOB=,40.753447012345966,=BLOB=,=BLOB=,0,975,0,5,nan,0.0,nan,2750,2329,7,48,0.9974610083424011,0.9828448892065761,0.99009900990099,429.31,864691135758569550,True,True,full,2,0,2,0,
88.00,258307,864691135293126156,0.08,0,=BLOB=,1074.2131095105929,=BLOB=,=BLOB=,43,340,12,27,0.7818181818181819,0.6142857142857143,0.6880000000000002,2984,7,25,19,0.9916915918909937,0.9936729936729937,0.9926813040585496,291.23,864691136194042326,True,True,full,1,0,0,0,
88.00,262773,864691136023889209,0.08,0,=BLOB=,191.70683649485437,=BLOB=,=BLOB=,9,48,2,0,0.8181818181818182,1.0,0.9,724,0,2,10,0.9972451790633609,0.9863760217983651,0.9917808219178083,88.75,864691135233108569,True,True,full,1,0,0,0,
88.00,264870,864691136951664863,0.08,0,=BLOB=,1021.2379299332463,=BLOB=,=BLOB=,31,422,2,1,0.9393939393939394,0.96875,0.9538461538461539,2008,25,4,33,0.9980119284294234,0.9838314551690348,0.9908709597828769,253.79,864691135740225387,True,True,full,1,0,0,0,
88.00,269247,864691135584074360,0.08,0,=BLOB=,2323.8059162243367,=BLOB=,=BLOB=,237,36,4,7,0.983402489626556,0.9713114754098361,0.9773195876288661,806,0,7,65,0.991389913899139,0.9253731343283582,0.9572446555819477,420.94,864691136105498585,True,True,full,1,0,0,0,
88.00,269380,864691135974639471,0.08,0,=BLOB=,437.8471079089947,=BLOB=,=BLOB=,27,257,4,11,0.8709677419354839,0.7105263157894737,0.782608695652174,1863,4,8,3,0.9957242116515232,0.9983922829581994,0.9970564624029972,249.06,864691134988386682,True,True,full,1,0,0,0,


In [7]:
curr_segment_id = 864691135865773189
search_key = dict(segment_id=curr_segment_id)
(curr_table & search_key)

ver materialization version,nucleus_id nucleus id,segment_id the segment id in the proofread version,old_ver materialization version,split_index the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM,decomposition saved neuron object with high fidelity axon,axon_length axon length of the filtered neuron,validation_df,validation_df_ext,pre_tp,pre_tn,pre_fp,pre_fn,pre_precision,pre_recall,pre_f1,post_tp,post_tn,post_fp,post_fn,post_precision,post_recall,post_f1,run_time the amount of time to run (seconds),old_segment_id the segment id in the proofread version--,finished_dendrite whether axon was finished cleaning,finished_axon whether axon was finished cleaning,extension_type,n_somas,n_soma_mergers,n_glia,large_splits,notes
88.00,493419,864691135865773189,0.08,0,=BLOB=,666.5064648944619,=BLOB=,=BLOB=,6,104,0,11,1.0,0.35294117647058826,0.5217391304347826,3883,4,8,4,0.9979439732716525,0.998970928736815,0.9984571869375161,340.42,864691136003930314,True,True,full,1,0,0,0,


In [8]:
split_index = 0
segment_id = curr_segment_id
decomp_path = (minnie.DecompositionAxon() & dict(segment_id=segment_id,
                                  split_index = split_index,
                                 )).fetch1("decomposition")
neuron_obj = du.filepath_to_neuron_obj(decomp_path)

Decompressing Neuron in minimal output mode...please wait


In [ ]:
not_stitched_pieces = nru.non_soma_touching_meshes_not_stitched(neuron_obj)

# Debugging the Stitching

In [12]:
import system_utils as su
main_limb_skeletons = su.decompress_pickle("main_limb_skeletons")
floating_limbs_skeleton_endpoints = su.decompress_pickle("floating_limbs_skeleton_endpoints")
floating_piece_min_distance_all_main_limbs = su.decompress_pickle("floating_piece_min_distance_all_main_limbs")
limb_correspondence_cp = su.decompress_pickle("limb_correspondence_cp")
excluded_meshes = su.decompress_pickle("excluded_meshes")
floating_limbs_skeleton = su.decompress_pickle("floating_limbs_skeleton")
floating_meshes_non_incorporated = su.decompress_pickle("floating_meshes_non_incorporated")

# Step 1): Look at the floating meshes excluded

In [16]:
nviz.plot_objects(main_mesh=neuron_obj.axon_mesh,
                meshes=excluded_meshes,
                 mesh_alpha=1,
                 meshes_colors="red")

In [15]:
nviz.plot_objects(meshes=excluded_meshes,
                 meshes_colors="purple",
                 append_figure=True,
                 show_at_end=False)

# Look at the skeleetons of the floating pieces

In [19]:
nviz.plot_objects(main_mesh=neuron_obj.axon_mesh,
                meshes=floating_meshes_non_incorporated,
                  meshes_colors="red",
                 skeletons=floating_limbs_skeleton,
                 scatters=floating_limbs_skeleton_endpoints)

In [20]:
nviz.plot_objects(main_mesh=neuron_obj.axon_mesh,
                meshes=floating_meshes_non_incorporated,
                  meshes_colors="red",
                 skeletons=floating_limbs_skeleton,
                 scatters=floating_limbs_skeleton_endpoints)

# Look at the limbs it says were not processed

In [21]:
floating_limbs_to_process = su.decompress_pickle("floating_limbs_to_process")
floating_limbs_above_threshold = su.decompress_pickle("floating_limbs_above_threshold")

In [23]:
still_not_processed = np.array(floating_limbs_above_threshold)[floating_limbs_to_process]
still_not_processed

array([<trimesh.Trimesh(vertices.shape=(480, 3), faces.shape=(949, 3))>,
       <trimesh.Trimesh(vertices.shape=(281, 3), faces.shape=(556, 3))>,
       <trimesh.Trimesh(vertices.shape=(221, 3), faces.shape=(440, 3))>,
       <trimesh.Trimesh(vertices.shape=(217, 3), faces.shape=(430, 3))>,
       <trimesh.Trimesh(vertices.shape=(206, 3), faces.shape=(249, 3))>,
       <trimesh.Trimesh(vertices.shape=(156, 3), faces.shape=(316, 3))>,
       <trimesh.Trimesh(vertices.shape=(133, 3), faces.shape=(258, 3))>,
       <trimesh.Trimesh(vertices.shape=(117, 3), faces.shape=(214, 3))>,
       <trimesh.Trimesh(vertices.shape=(114, 3), faces.shape=(200, 3))>,
       <trimesh.Trimesh(vertices.shape=(109, 3), faces.shape=(214, 3))>,
       <trimesh.Trimesh(vertices.shape=(98, 3), faces.shape=(102, 3))>,
       <trimesh.Trimesh(vertices.shape=(77, 3), faces.shape=(149, 3))>,
       <trimesh.Trimesh(vertices.shape=(75, 3), faces.shape=(146, 3))>,
       <trimesh.Trimesh(vertices.shape=(74, 3), faces.

In [25]:
floating_limbs_to_process

array([ 50,  60,  64,  65,  66,  71,  74,  76,  78,  80,  86,  90,  92,
        93,  95,  97,  99, 100])

In [26]:
nviz.plot_objects(main_mesh=neuron_obj.axon_mesh,
                meshes=floating_limbs_above_threshold,
                  meshes_colors="red",
                 skeletons=floating_limbs_skeleton,
                 scatters=floating_limbs_skeleton_endpoints)

In [29]:
nviz.plot_objects(main_mesh=neuron_obj.axon_mesh,
                meshes=still_not_processed,
                  meshes_colors="red",
                 
                  skeletons=np.array(floating_limbs_skeleton)[floating_limbs_to_process],
                 scatters=floating_limbs_skeleton_endpoints)

# Check that the limb correspondence has everything

In [36]:
import trimesh_utils as tu
non_touch_meshes = tu.combine_meshes(neuron_obj.non_soma_touching_meshes)
non_incorp_meshes = tu.combine_meshes(floating_meshes_non_incorporated)


In [40]:
nviz.plot_objects(tu.subtract_mesh(tu.subtract_mesh(non_touch_meshes,non_incorp_meshes),neuron_obj.axon_mesh,exact_match=False))

In [31]:
meshes,skeletons = nviz.limb_correspondence_plottable(limb_correspondence_cp)
nviz.plot_objects(skeletons=skeletons,
                 meshes=floating_meshes_non_incorporated,
                 mesh_alpha=1,
                 meshes_colors="red")

In [33]:
meshes,skeletons = nviz.limb_correspondence_plottable(limb_correspondence_cp)
nviz.plot_objects(skeletons=skeletons,
                 meshes=neuron_obj.non_soma_touching_meshes,
                 mesh_alpha=0.5,
                 meshes_colors="red")

In [46]:
processed_axon_mesh = su.decompress_pickle("processed_axon_mesh")
processed_axon_skeleton = su.decompress_pickle("processed_axon_skeleton")

In [49]:
limb_correspondence_cp.keys()

dict_keys([0])

In [47]:
nviz.plot_objects(meshes=meshes,
                  meshes_colors="red",
                 skeletons=[processed_axon_skeleton])

In [50]:
import axon_utils as au
neuron_obj_axon = au.complete_axon_processing(neuron_obj)

endnodes_to_test = []
endnodes_to_test = []
endnodes_to_test = [17181]
Skeleton paths distances = [24930.07858738]
Filtered indexes = [0]
len(filtered_skeletons) = 1
sk_angles = [147.62416683]
base_final_widths = [190.71999881 191.59622564 227.55154207]
overall_ais_width = 190.71999880599452
ais_width = 203.28925550568695
endnodes_to_test = [0]
Skeleton paths distances = [2459.1975387]
Filtered indexes = []
len(filtered_skeletons) = 1
sk_angles = [129.12507118]
base_final_widths = [117.78919948 438.43601345]
overall_ais_width = 278.1126064632469
ais_width = 278.1126064632469
endnodes_to_test = [0]
Skeleton paths distances = [2667.79972703]
Filtered indexes = []
len(filtered_skeletons) = 1
sk_angles = [78.5900486]
endnodes_to_test = [0]
Skeleton paths distances = [4157.27246412]
Filtered indexes = []
len(filtered_skeletons) = 1
sk_angles = [112.43574784]
base_final_widths = [144.08153957 552.41597278]
overall_ais_width = 348.24875617643903
ais_width = 348.24875617643903
endnodes_to_test

  0%|          | 0/1 [00:00<?, ?it/s]

More than 1 seperated mesh (taking the top one): [<trimesh.Trimesh(vertices.shape=(110913, 3), faces.shape=(221205, 3))>
 <trimesh.Trimesh(vertices.shape=(10311, 3), faces.shape=(20439, 3))>
 <trimesh.Trimesh(vertices.shape=(8175, 3), faces.shape=(16316, 3))>
 <trimesh.Trimesh(vertices.shape=(8031, 3), faces.shape=(15947, 3))>
 <trimesh.Trimesh(vertices.shape=(7697, 3), faces.shape=(15403, 3))>
 <trimesh.Trimesh(vertices.shape=(2579, 3), faces.shape=(5145, 3))>
 <trimesh.Trimesh(vertices.shape=(1932, 3), faces.shape=(3860, 3))>
 <trimesh.Trimesh(vertices.shape=(1376, 3), faces.shape=(2751, 3))>
 <trimesh.Trimesh(vertices.shape=(831, 3), faces.shape=(1662, 3))>
 <trimesh.Trimesh(vertices.shape=(630, 3), faces.shape=(1260, 3))>
 <trimesh.Trimesh(vertices.shape=(278, 3), faces.shape=(556, 3))>]
root_curr = [1170227.   686364.   890919.7]
Time for preparing soma vertices and root: 0.0004029273986816406
invalidation_d = 1500
smooth_neighborhood = 0
cc_vertex_thresh = 10


  0%|          | 0/110912 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:02<00:00,  2.60s/it]


...

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/44 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

  0%|          | 0/39 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

In [51]:
print("hi")

hi


In [53]:
nviz.visualize_neuron(neuron_obj_axon,
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton
